In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Gemini 2.0 Flash



## Overview


[Gemini 2.0 Flash](https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2) is a new multimodal generative ai model from the Gemini family developed by [Google DeepMind](https://deepmind.google/). It is available through the Gemini API in Vertex AI and Vertex AI Studio. The model introduces new features and enhanced core capabilities:

- Multimodal Live API: This new API helps you create real-time vision and audio streaming applications with tool use.
- Speed and performance: Gemini 2.0 Flash is the fastest model in the industry, with a 3x improvement in time to first token (TTFT) over 1.5 Flash.
- Quality: The model maintains quality comparable to larger models like Gemini 2.0 and GPT-4o.
- Improved agentic experiences: Gemini 2.0 delivers improvements to multimodal understanding, coding, complex instruction following, and function calling.
- New Modalities: Gemini 2.0 introduces native image generation and controllable text-to-speech capabilities, enabling image editing, localized artwork creation, and expressive storytelling.
- To support the new model, we're also shipping an all new SDK that supports simple migration between the Gemini Developer API and the Gemini API in Vertex AI.

### Objectives

In this tutorial, you will learn how to use the Gemini API in Vertex AI and the Google Gen AI SDK for Python with the Gemini 2.0 Flash model.

You will complete the following tasks:

- Generate text from text prompts
  - Start multi-turn chats
- Configure model parameters
- Set system instructions
- Process multimodal (audio, code, documents, images, video) data

## Getting Started

### Install Google Gen AI SDK for Python


In [3]:
%pip install --upgrade --quiet google-genai

### Connect to a generative AI API service

Google Gen AI APIs and models including Gemini are available in the following two API services:

- **[Google AI for Developers](https://ai.google.dev/gemini-api/docs)**: Experiment, prototype, and deploy small projects.
- **[Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview)**: Build enterprise-ready projects on Google Cloud.

The Google Gen AI SDK provides a unified interface to these two API services.

This notebook shows how to use the Google Gen AI SDK with the Gemini API in Vertex AI.

### Import libraries


In [4]:
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    MediaResolution,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    ToolCodeExecution,
    VertexAISearch,
)

### Set up Google Cloud Project 


This requires enabling the Vertex AI API in your Google Cloud project.
    [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
    *   Run the cell below to set your project ID.


In [5]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Use the Gemini 2.0 Flash model

### Load the Gemini 2.0 Flash model

Learn more about all [Gemini models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

In [7]:
MODEL_ID = "gemini-2.0-flash"  # @param {type: "string"}

### Generate text from text prompts

Use the `generate_content()` method to generate responses to your prompts.

You can pass text to `generate_content()`, and use the `.text` property to get the text content of the response.

By default, Gemini outputs formatted text using [Markdown](https://daringfireball.net/projects/markdown/) syntax.

In [8]:
response = client.models.generate_content(
    model=MODEL_ID, contents="What's the largest planet in our solar system?"
)

display(Markdown(response.text))

#### Example prompts

- What are the biggest challenges facing the healthcare industry?
- What are the latest developments in the automotive industry?
- What are the biggest opportunities in retail industry?
- (Try your own prompts!)

For more examples of prompt engineering, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb).

### Start a multi-turn chat

The Gemini API supports freeform multi-turn conversations across multiple turns with back-and-forth interactions.

The context of the conversation is preserved between messages.

In [ ]:
chat = client.chats.create(model=MODEL_ID)

In [ ]:
response = chat.send_message("Write a function that checks if a year is a leap year.")

display(Markdown(response.text))

This follow-up prompt shows how the model responds based on the previous prompt:

In [ ]:
response = chat.send_message("Write a unit test of the generated function.")

display(Markdown(response.text))

## Configure model parameters

You can include parameter values in each call that you send to a model to control how the model generates a response. The model can generate different results for different parameter values. You can experiment with different model parameters to see how the results change.

- Learn more about [experimenting with parameter values](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/adjust-parameter-values).

- See a list of all [Gemini API parameters](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#parameters).


In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Tell me how the internet works, but pretend I'm a puppy who only understands squeaky toys.",
    config=GenerateContentConfig(
        temperature=0.4,
        top_p=0.95,
        top_k=20,
        candidate_count=1,
        seed=5,
        max_output_tokens=100,
        stop_sequences=["STOP!"],
        presence_penalty=0.0,
        frequency_penalty=0.0,
    ),
)

display(Markdown(response.text))

## Set system instructions

[System instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instruction-introduction) allow you to steer the behavior of the model. By setting the system instruction, you are giving the model additional context to understand the task, provide more customized responses, and adhere to guidelines over the user interaction.

In [ ]:
system_instruction = """
  You are a helpful language translator.
  Your mission is to translate text in English to Spanish.
"""

prompt = """
  User input: I like bagels.
  Answer:
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

## Send multimodal prompts

Gemini is a multimodal model that supports multimodal prompts.

You can include any of the following data types from various sources.

<table>
  <thead>
    <tr>
      <th>Data type</th>
      <th>Source(s)</th>
      <th>MIME Type(s)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Text</td>
      <td>Inline, Local File, General URL, Google Cloud Storage</td>
      <td><code>text/plain</code></td>
    </tr>
    <tr>
      <td>Code</td>
      <td>Inline, Local File, General URL, Google Cloud Storage</td>
      <td><code>text/plain</code></td>
    </tr>
    <tr>
      <td>Document</td>
      <td>Local File, General URL, Google Cloud Storage</td>
      <td><code>application/pdf</code></td>
    </tr>
    <tr>
      <td>Image</td>
      <td>Local File, General URL, Google Cloud Storage</td>
      <td><code>image/jpeg</code> <code>image/png</code> <code>image/webp</code></td>
    </tr>
    <tr>
      <td>Audio</td>
      <td>Local File, General URL, Google Cloud Storage</td>
      <td>
        <code>audio/aac</code> <code>audio/flac</code> <code>audio/mp3</code>
        <code>audio/m4a</code> <code>audio/mpeg</code> <code>audio/mpga</code>
        <code>audio/mp4</code> <code>audio/opus</code> <code>audio/pcm</code>
        <code>audio/wav</code> <code>audio/webm</code>
      </td>
    </tr>
    <tr>
      <td>Video</td>
      <td>Local File, General URL, Google Cloud Storage, YouTube</td>
      <td>
        <code>video/mp4</code> <code>video/mpeg</code> <code>video/x-flv</code>
        <code>video/quicktime</code> <code>video/mpegps</code> <code>video/mpg</code>
        <code>video/webm</code> <code>video/wmv</code> <code>video/3gpp</code>
      </td>
    </tr>
  </tbody>
</table>

Set `config.media_resolution` to optimize for speed or quality. Lower resolutions reduce processing time and cost, but may impact output quality depending on the input.

For more examples of multimodal use cases, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb).

### Send local image

Download an image to local storage from Google Cloud Storage.

For this example, we'll use this image of a meal.

<img src="https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png" alt="Meal" width="500">

In [ ]:
!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png

In [ ]:
with open("meal.png", "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        "Write a short and engaging blog post based on this picture.",
    ],
    # Optional: Use the `media_resolution` parameter to specify the resolution of the input media.
    config=GenerateContentConfig(
        media_resolution=MediaResolution.MEDIA_RESOLUTION_LOW,
    ),
)

display(Markdown(response.text))

### Send document from Google Cloud Storage

This example document is the paper ["Attention is All You Need"](https://arxiv.org/abs/1706.03762), created by researchers from Google and the University of Toronto.

Check out this notebook for more examples of document understanding with Gemini:

- [Document Processing with Gemini](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/document_processing.ipynb)

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="gs://cloud-samples-data/generative-ai/pdf/1706.03762v7.pdf",
            mime_type="application/pdf",
        ),
        "Summarize the document.",
    ],
)

display(Markdown(response.text))

### Send audio from General URL

This example is audio from an episode of the [Kubernetes Podcast](https://kubernetespodcast.com/).

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3",
            mime_type="audio/mpeg",
        ),
        "Write a summary of this podcast episode.",
    ],
    config=GenerateContentConfig(audio_timestamp=True),
)

display(Markdown(response.text))

### Send video from YouTube URL

This example is the YouTube video [Google — 25 Years in Search: The Most Searched](https://www.youtube.com/watch?v=3KtWfp0UopM).


In [ ]:
video = Part.from_uri(
    file_uri="https://www.youtube.com/watch?v=3KtWfp0UopM",
    mime_type="video/mp4",
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        "At what point in the video is Harry Potter shown?",
    ],
)

display(Markdown(response.text))

## Provisioned Throughput

For high-scale production use cases, [Provisioned Throughput](https://cloud.google.com/vertex-ai/generative-ai/docs/provisioned-throughput) allows for reserved capacity of generative AI models on Vertex AI.

Once you have it [set up for your project](https://cloud.google.com/vertex-ai/generative-ai/docs/purchase-provisioned-throughput), refer to [Use Provisioned Throughput](https://cloud.google.com/vertex-ai/generative-ai/docs/use-provisioned-throughput) for usage instructions.

## What's next

- See the [Google Gen AI SDK reference docs](https://googleapis.github.io/python-genai/).
- Explore other notebooks in the [Google Cloud Generative AI GitHub repository](https://github.com/GoogleCloudPlatform/generative-ai).
- Explore AI models in [Model Garden](https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models).